**Start with your name(s) and student number(s)**

Team:

* FILL IN YOUR NAME (AND STUDENT NUMBER)

# Outline

In this notebook we will learn about **parameter learning** in Probabilistic Graphical Models (PGMs). The main goal is to understand how we can estimate the parameters of a PGM from data.

We will use Hidden Markov Models (HMMs) as a concrete example to illustrate different learning algorithms. HMMs are a special type of Bayesian network that model sequences, but the parameter estimation techniques we'll learn apply more broadly to PGMs.

This is a high-level outline of the notebook, you will find exercises in most sections.

1. **HMMs and Foward Sampling**: We introduce HMMs as an example model (briefly, just enough to understand the task)
2. **Training with MLE**: We learn how to estimate parameters using Maximum Likelihood Estimation (MLE) via count-and-normalize
3. **Testing**: We evaluate the trained model on test data using:
   - Max-product inference from text and classification reports
   - Reconstructing missing POS tags
   - Computing marginal probabilities for fluency ranking
4. **Semi-supervised Learning**: We learn parameters using a combination of labeled and unlabeled data

**Table of Exercises**

The exercises and the points they are worth are shown below:

1. Exercise - Implement Forward Sampling [1pt]
2. Exercise - Understanding MLE [0pt]
3. Exercise - Implementing MLE [2pt]
4. Exercise - Inferring POS-tags from text - Code [1pt]
5. Exercise - Analysis POS-tagging [1pt]
6. Exercise - Reconstruct Missing POS Tags [2pts]
7. Exercise - Rank Fluent vs Disfluent Sentences - Code [1pt]
8. Exercise - Rank Fluent vs Disfluent Sentences - Analysis [1pt]
9. Exercise - Semi-supervised Learning [1pts]

There are 10 points above.

**Use of AI tools**

In this course we expect _you_ and your team members to author your work.
AI tools are not to be used for drafts, nor code completion, nor revisions, nor as a source of feedback. If you do use AI, it should not contribute to the substance of what you present as your work.  

At the end of this notebook you will find a section on _Use of AI tools_. **Make sure to read and complete it**. 
By submitting a version of this notebook for assessment, you agree with our terms.

# Setting Up

Take care of dependencies:


In [ ]:
# !pip install tabulate
# !pip install --upgrade --force-reinstall  git+https://github.com/probabll/pgmini.git
# !pip install scikit-learn
# !pip install tqdm

In [ ]:
import pgmini
# assert pgmini.__version__ == '0.5.0', "Don't forget to update pgmini and restart your kernel"


In [ ]:
from pgmini.m1 import OutcomeSpace
from pgmini.m3 import TabularCPDFactor, BayesianNetwork, MarkovNetwork
from pgmini.m4 import max_product_variable_elimination, sum_product_variable_elimination, split_factors
from pgmini.m5 import gibbs_sampler, draw_initial_assignment
from pgmini.util import make_samples_df, df_to_factor, tvd, pgm_to_tabular_factor
import numpy as np
from tabulate import tabulate
import pickle
import functools
from collections import OrderedDict
import itertools
from sklearn.metrics import classification_report
from tqdm.auto import tqdm


# Hidden Markov Models (HMMs) as an Example

To make the learning process concrete, we'll use **Hidden Markov Models (HMMs)** as our example. HMMs are a special type of Bayesian network that model sequences where we observe one sequence and want to predict another.

## What is an HMM?

An HMM models the joint distribution of two sequences of the same length:
- **Observed sequence** $X = (x_1, x_2, \ldots, x_n)$: what we can see
- **Hidden sequence** $Y = (y_1, y_2, \ldots, y_n)$: what we want to predict

For example, in part-of-speech (POS) tagging:
- $X$ is a sequence of words: "the cat sat"
- $Y$ is a sequence of POS tags: "DET NOUN VERB"

The HMM makes two key assumptions:
1. **Markov assumption**: Each hidden state $H_i$ depends only on the previous hidden state $H_{i-1}$
2. **Independence assumption**: Each observation $O_i$ depends only on the corresponding hidden state $H_i$

The joint probability is:
$$P(X=x, Y=y) = \prod_{i=1}^n P(H_i=y_i|H_{i-1}=y_{i-1}) P(O_i=x_i|H_i=y_i)$$

Where $n = |x|$ is the length of the sequence. We assume the parent of the first hidden variable $H_1$ is always some fixed value such as $H_0=h^0$.

## Parameter Sharing

HMMs use **parameter sharing**: all transitions $P(H_i|H_{i-1})$ share the same parameters (a transition table), and all emissions $P(O_i|H_i)$ share the same parameters (an emission table). This makes learning more efficient and allows the model to generalize to sequences of any length.


## Constructing an HMM

Let's see how to construct an HMM as a Bayesian network. 

To construct HMMs we will use templates. These templates are conditional probability tables that we repeat for the amount of steps in a HMM (this is parameter sharing!).

The function below builds a BN for a sequence of a given length using these templates:


In [ ]:
def construct_hmm(hidden_rv: str, obs_rv: str, outcome_spaces: dict, h_bos: str, templates: dict, length: int):
    """
    Return a BN representation of the HMM distribution over sequence pairs of a certain length.
    
    hidden_rv: name of the hidden rv H (e.g, H for 'hidden')
    obs_rv: name of the observed rv O (e.g., O for 'observed')
    outcome_spaces: dict mapping an rv (hidden or obs) to its OutcomeSpace object
    h_bos: the outcome in Val(H) which is to be used to denote that a hidden sequence has started
    templates: dict mapping an rv (hidden or obs) to its template tabular CPD
    length: the length of the sequence pair
    """
    assert 0 < length < 1000, "In this lab we will not be modelling sequences longer than 999 steps."
    
    factors = []
    
    val_h = outcome_spaces[hidden_rv]
    val_o = outcome_spaces[obs_rv]
        
    # H[001] - first hidden state (depends on BOS)
    factors.append(TabularCPDFactor([], f"{hidden_rv}[001]", {f"{hidden_rv}[001]": val_h}, values=templates[hidden_rv][val_h[h_bos]]))

    # H[i] -> H[i+1] - transitions
    for j in range(2, length+1):
        i = j - 1
        hi = f'{hidden_rv}[{i:03d}]'
        hj = f'{hidden_rv}[{j:03d}]'
        phi_H = TabularCPDFactor([hi], hj, {hi: val_h, hj: val_h}, values=None, from_template=templates[hidden_rv])
        factors.append(phi_H)

    # H[i] -> O[i] - emissions
    for k in range(1, length+1):
        hk = f'{hidden_rv}[{k:03d}]' 
        ok = f'{obs_rv}[{k:03d}]'
        phi_O = TabularCPDFactor([hk], ok, {hk: val_h, ok: val_o}, values=None, from_template=templates[obs_rv])
        factors.append(phi_O)

    return BayesianNetwork(factors)


## Example: A Simple HMM with Two Hidden States

Let's build a simple HMM to illustrate how these models work. We'll define a model with two hidden states (0 and 1) and observations that are digits 0-9. Beginning of sentence (BOS) and end of sentence (EOS) are just tools to let us start and finish the chain.

**Transition probabilities:**
- For the hidden state at time $i=0$, we have:
  - $P(H_1 = 0 | H_0 = \text{BOS}) = 0.5$
  - $P(H_1 = 1 | H_0 = \text{BOS}) = 0.5$
- If the hidden state at time $i$ is 0 ($H_i = 0$), then:
  - $P(H_{i+1} = 0 | H_i = 0) = 0.8$ (stays in state 0)
  - $P(H_{i+1} = 1 | H_i = 0) = 0.1$ (transitions to state 1)
  - $P(H_{i+1} = \text{EOS} | H_i = 0) = 0.1$ (ends the sequence)
- Similarly, if $H_i = 1$:
  - $P(H_{i+1} = 1 | H_i = 1) = 0.8$ (stays in state 1)
  - $P(H_{i+1} = 0 | H_i = 1) = 0.1$ (transitions to state 0)
  - $P(H_{i+1} = \text{EOS} | H_i = 1) = 0.1$ (ends the sequence)

**Emission probabilities:**
- If the hidden state is 0 ($H_i = 0$), the emission probability $P(O_i | H_i = 0)$ is a uniform distribution over all **even** numbers in 0-9: $\{0, 2, 4, 6, 8\}$
- If the hidden state is 1 ($H_i = 1$), the emission probability $P(O_i | H_i = 1)$ is a uniform distribution over all **odd** numbers in 0-9: $\{1, 3, 5, 7, 9\}$

Note that the model requires us to define conditional distibutions for all variables (both transitions and emissions), so we will add '-BOS-' and '-EOS-' as emissions for '-BOS-' and '-EOS-' respectively. Also, if '-EOS-' is sampled as the outcome of the hidden state, all next outcomes will also be '-EOS-'.

Let's generate this model and verify that it works with sampling.

First we start with building the conditional probability tables:

In [ ]:
# Define outcome spaces for the simple HMM example
simple_outcome_spaces = {
    'H': OutcomeSpace(['-BOS-', '0', '1', '-EOS-']),  # Hidden states: BOS, 0, 1, EOS
    'O': OutcomeSpace(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-BOS-', '-EOS-']),  # Observations: digits 0-9
}

# Initialize transition probability table
transitions = np.zeros((4, 4))
transitions[0, :] = np.array([[0, .5, .5, 0]]) # Transition from BOS to 0 or 1
transitions[1, :] = np.array([[0, .8, .1, .1]]) # Transition from 0 to 0, 1, EOS
transitions[2, :] = np.array([[0, .1, .8, .1]]) # Transition from 1 to 0, 1, EOS
transitions[3, :] = np.array([[0, 0, 0, 1]]) # EOS is absorbing

# Initialize emission probability table
emissions = np.zeros((4, 12))
emissions[0, -2] = 1.0 # BOS emits BOS
emissions[1, :] = np.array([[.2, 0, .2, 0, .2, 0, .2, 0, .2, 0, 0, 0]]) # State 0 emits even numbers uniformly
emissions[2, :] = np.array([[0, .2, 0, .2, 0, .2, 0, .2, 0, .2, 0, 0]]) # State 1 emits odd numbers uniformly
emissions[3, -1] = 1.0 # EOS emits EOS

# Create template factors
simple_templates = {
    'H': transitions,
    'O': emissions
}

# Verify that rows sum to 1 (each row defines all possible outcomes for a given condition)
assert np.sum(emissions, axis=-1).all() == 1.0
assert np.sum(transitions, axis=-1).all() == 1.0

print("Transition probabilities (rows: from state, cols: to state):")
print(f"Hidden states: \n[-BOS-, 0, 1, -EOS-]: \n{transitions}")
print("\nEmission probabilities (rows: hidden state, cols: observed digit):")
print(f"Observed digits: \n[0, 1, 2, 3, 4, 5, 6, 7, 8, 9,-BOS-, -EOS-]: \n{emissions}")

## Forward Sampling

Now that we have defined a model, we need a way to sample from it. As we have a BN without evidence we can just use forward sampling. 

Make sure to loop over the graph in topologiical order!


 **EXERCISE - Implement Forward Sampling [1pt]**
Complete the function `forward_sampling()` and check the results.

In [ ]:
def forward_sampling(bn: BayesianNetwork, rng=np.random.default_rng(42)):
    """
    Forward sampling from a BN
    Loop over the variables in the BN in topological order, 
    sampling the outcome for each variable conditioned on its parents.

    :param bn: a BayesianNetwork object
    :param rng: a random number generator
    :return: a joint assignment of the variables in the BN
    """

    ### YOUR CODE HERE


    raise NotImplementedError("You need to implement this!")


Let's define some helper functions to visualize the joint assignments:

In [ ]:
def tostring(seq_pair, vertical=True, headers=['Observed', 'Hidden']):
    """
    :param seq_pair: a sequence of pairs for (O, H) outcomes.
    :param vertical: use True for vertical printing with tabulate.
    :return: a string representing the sequence of pairs.
    """
    if vertical:
        return tabulate(list(seq_pair), headers=headers)
    else:
        return ' '.join(f'{o}/{h}' for o, h in seq_pair)

def tostring2(obs_seq, hid_seq, vertical=True, headers=['Observed', 'Hidden']):
    """
    :param obs_seq: a sequence of observations
    :param hid_seq: a corresponding sequence of hidden labels
        they must have the same length
    :param vertical: use True for vertical printing with tabulate.
    :return: a string representing the sequence of pairs.
    """
    assert len(obs_seq) == len(hid_seq), "I need the same number of elements in both sequences"
    return tostring(zip(obs_seq, hid_seq), vertical=vertical, headers=headers)

def assignment_tostring(assignment, obs_rv, hidden_rv, vertical=True):    
    return tostring2([assignment[rv] for rv in sorted(assignment.keys()) if rv.startswith(obs_rv)], 
                     [assignment[rv] for rv in sorted(assignment.keys()) if rv.startswith(hidden_rv)], 
                     vertical=vertical)



Now let's visualize some examples:

In [ ]:
# Construct the BN
# We will construct BNs up to length 10
simple_bn = construct_hmm('H', 'O', simple_outcome_spaces, '-BOS-', simple_templates, 10)
# Sample a few sequences
print("Sampling sequences from the simple HMM:")
rng = np.random.default_rng(42)
for i in range(3):
    sample = forward_sampling(simple_bn, rng=rng)   
    for rv, observed in sample.copy().items():
        # Lets remove the -EOS- tokens
        if observed == '-EOS-':
            sample.pop(rv)
            
    # Use assignment_tostring to display the sample
    print(f"\nSample {i+1}:")
    print(assignment_tostring(sample, 'O', 'H'))


# Training: Maximum Likelihood Estimation (for Discrete PGMs)

Now that we know what HMMs are and how we can sample from them, the next exercises will be about how we can learn the parameters of an HMM.

## A Brief Reminder: How to Calculate MLE

Given an outcome for an input variable ($H_{i}=y_m$):
- Count the number of outcomes of each output variable $\#(H_{i+1}=y_m|H_{i}=y_n)$ or $\#(X_i=x_m|H_{i}=y_n)$ 
- Divide by the total count

Where $X$ and $Y$ are the outcome spaces and $y_m, y_n \in Y$ and $x_m \in X$ are outcomes.

Do this for both transitions ($P(H_{t+1}|H_{t})$) and emissions ($P(X_{t}|H_{t})$).

**EXERCISE - Understanding MLE [0pts] (but still good to do)**

Before implementing MLE, let's understand what we're counting.

**Question**: In an HMM, if we have training data with sequences like:
- (words: "the cat", tags: "DET NOUN")
- (words: "a dog", tags: "DET NOUN")

In this case, the tags are the outcomes of the hidden variables and the words are the observed outcomes.

What transition counts would we observe? What emission counts?

**ANSWER:**

- Transition counts: DET→NOUN appears 2 times
- Emission counts: (DET, "the") appears 1 time, (DET, "a") appears 1 time, (NOUN, "cat") appears 1 time, (NOUN, "dog") appears 1 time


**EXERCISE - Implement MLE [2 points]**

Finish the function `estimate_parameters_hmm()` to get CPD tables that represent the Maximum Likelihood Estimation given some dataset of pairs.

To count transitions, we define a helper function to iterate over consecutive pairs in sequences (you might find it useful):

In [ ]:
def iter_bigrams(seq):
    """Iterate over consecutive pairs in a sequence"""
    a, b = itertools.tee(seq, 2)
    next(b, None)    
    return zip(a, b)


In [ ]:
def estimate_parameters_hmm(pairs, hidden_rv, obs_rv, outcome_spaces):
    """
    Estimate HMM parameters using MLE.
    - pairs: training data
    - hidden_rv: name of hidden rv
    - obs_rv: name of observed rv
    outcome_spaces: dict mapping rv name to its OutcomeSpace
    
    Returns a dict with:
    - hidden_rv: transition probability table [num_hidden_states, num_hidden_states]
    - obs_rv: emission probability table [num_hidden_states, num_obs_states]
    """
    val_h = outcome_spaces[hidden_rv]
    val_o = outcome_spaces[obs_rv]
    
    # Initialize count tables
    transitions = np.zeros([len(val_h), len(val_h)])
    emissions = np.zeros([len(val_h), len(val_o)])

    ### YOUR CODE HERE
    # for x, y in pairs:
        

    raise NotImplementedError("You need to implement this!")



## Comparing to Ground-Truth Model

We can test our MLE implementation by building on the methods from last week:

1. Generate data from a known model
2. Use that data to train the model 
3. Compare the trained model to the 'true' model

### Generate Data

Note that we add a '-BOS-' token to each sequence so we can learn to generate correctly when sampling from the '-BOS-' token.

In [ ]:
# Generate multiple sequences
num_samples = 1000
generated_pairs = []

for sample_idx in range(num_samples):
    # Sample a sequence from the model
    sample = forward_sampling(simple_bn, rng=rng)

    # Add the -BOS- token
    sample['H[000]'] = sample['O[000]'] = '-BOS-'

    # Extract hidden and observed sequences
    hidden_seq = [sample[rv] for rv in sorted([rv for rv in sample.keys() if rv.startswith('H')])]
    obs_seq = [sample[rv] for rv in sorted([rv for rv in sample.keys() if rv.startswith('O')])]         
    generated_pairs.append((obs_seq, hidden_seq))

print(f"Generated {len(generated_pairs)} sequence pairs")
print(f"Average sequence length: {np.mean([len(x) for x, y in generated_pairs]):.1f}\n")

# Show a few examples
print("Example generated sequences (including -BOS- as the first observation):")
for i in range(3):
    obs_seq, hidden_seq = generated_pairs[i]
    print(f"\nSequence {i+1}:")
    print(obs_seq)
    print(hidden_seq)


### Train the Model

In [ ]:
# Train the model on the generated data
learned_params = estimate_parameters_hmm(
    generated_pairs,
    'H',
    'O',
    simple_outcome_spaces
)
print(f"Learned Transition Probabilities:\n{np.round(learned_params['H'], 2)}")
print(f"Learned Emission Probabilities:\n{np.round(learned_params['O'], 2)}")

### Compare to Ground Truth

We can use the TVD method from last week to compare the results. We use our learned parameters to define a new HMM (which is just a chain of CPDs) and compare the TVD between its tables. For this, let's define BNs of length 1 with our new parameters:

In [ ]:
true_small_hmm = construct_hmm('H', 'O', simple_outcome_spaces, '-BOS-', simple_templates, 1)
learned_small_hmm = construct_hmm('H', 'O', simple_outcome_spaces, '-BOS-', learned_params, 1)

# Show hmm
print(true_small_hmm.dag)
true_small_hmm_table = pgm_to_tabular_factor(true_small_hmm)
learned_small_hmm_table = pgm_to_tabular_factor(learned_small_hmm)

print(f"TVD between true and learned small HMM: {tvd(true_small_hmm_table, learned_small_hmm_table):.4f}")

As you increase the number of samples, the TVD should get closer to 0.

## Real Data: POS Tagging

Now let's train an HMM on real data! We'll use a part-of-speech (POS) tagging dataset where:
- **Observed**: sequences of words
- **Hidden**: sequences of POS tags (NOUN, VERB, DET, etc.)

In part-of-speech (POS) tagging, our observed sequence $X$ is a sequence of words, each random word $W$ is a symbol in the vocabulary $\text{Val}(W)$ of a language (like English), the hidden sequence $Y$ is a sequence of POS tags (or syntactic word categories), each random tag $T$ is a symbol in a POS tagset $\text{Val}(T)$, this includes things like NOUN, VERB, DET, PRON, ADVERB, etc.


Note that now we do not have a ground-truth model. 

We do still want to have a model that is close to it, such that it generates well on other data (the validation- and test-set).

Let's load the data:


In [ ]:
with open("pos-english.pkl", "rb") as f:
    pos_dataset = pickle.load(f)

train_size = len(pos_dataset['training_x'])
dev_size = len(pos_dataset['dev_x'])
print("Dataset keys:", pos_dataset.keys())
print("\nNumber of training examples:", train_size)
print("Number of dev examples:", dev_size)


Let's look at a few examples:


In [ ]:
print("A few training examples:\n")
for n in range(3):    
    print(tostring2(pos_dataset['training_x'][n], pos_dataset['training_y'][n]))
    print()

Now let's set up the outcome spaces and train the model:


In [ ]:
# Set up outcome spaces
pos_outcome_spaces = {
    'T': OutcomeSpace(pos_dataset['vocab_t']),
    'W': OutcomeSpace(pos_dataset['vocab_w']),
}

# Train the model using MLE
pos_template_factors = estimate_parameters_hmm(
    zip(pos_dataset['training_x'], pos_dataset['training_y']),
    'T',
    'W',
    pos_outcome_spaces
)

# Some quick tests
assert pos_template_factors['T'].shape == (14, 14)
assert pos_template_factors['W'].shape == (14, 3315)
assert np.allclose(pos_template_factors['T'].sum(-1), 1.)
assert np.allclose(pos_template_factors['W'].sum(-1), 1.)


Similar to the example earlier, we can now construct a model and sample from it:

In [ ]:
pos_bn = construct_hmm('T', 'W', pos_outcome_spaces, '-BOS-', pos_template_factors, 10)
sample = forward_sampling(pos_bn, rng=rng)
print(assignment_tostring(sample, 'W', 'T'))

# Testing: Evaluating the Trained Model

Now that we have a trained model, let's evaluate it on validation data. We will need to show the performance of the model in a task that we care about.

We will define a couple of task, which you will have to implement, namely:
- Inferring POS-tags given a text sequence
- Inferring missing tags of a tagged sequence
- Comparing fluent versus non-fluent sequences



## Max-Product Inference for Prediction and Testing

The typical use of an HMM is to perform some inference task _given_ an observed sequence. We develop some helper code for building a BN that is appropriate for conditioning on a given sequence:

We'll use max-product inference to predict the most likely tag sequence for an observed word sequence. 

We have implemented a helper function that defines a BN for texts of any size.


In [ ]:
def make_evidence_from_text(obs_seq: list, hidden_rv: str, obs_rv: str, outcome_spaces: dict, h_bos: str, o_eos: str, o_unk: str, templates: dict):
    """
    Construct a BN with the correct number of variables and an evidence assignment.
    
    Returns:
    - bn: BayesianNetwork for the sequence length
    - e: OrderedDict with evidence assignments for observed variables
    """
    if obs_seq[-1] != o_eos:  # add o_eos if not there
        obs_seq = obs_seq + [o_eos]
    
    # Construct a BN for the right length
    bn = construct_hmm(hidden_rv, obs_rv, outcome_spaces, h_bos, templates, len(obs_seq))
    
    # Construct an assignment of the observed rvs
    e = OrderedDict([(f'{obs_rv}[{i:03d}]', obs if obs in outcome_spaces[obs_rv] else o_unk) 
                     for i, obs in enumerate(obs_seq, 1)])
    return bn, e


**EXERCISE - Inferring POS-tags from text [1pt]**

Implement the function `tag_seq_from_text()` below and check the results.


In [ ]:
def tag_seq_from_text(obs_seq: list, hidden_rv: str, obs_rv: str, outcome_spaces: dict, h_bos: str, o_eos: str, o_unk: str, templates: dict):
    """
    Predict the most likely tag sequence for an observed word sequence.
    returns:
    - pred: dict with predicted tags
    - e: OrderedDict with evidence assignments for observed variables
    """
    ### YOUR CODE HERE


    raise NotImplementedError("You need to implement this!")

Let's show the results:

In [ ]:
for n in [0, 1, 2, 3]:
    table = []
    pred, e = tag_seq_from_text(pos_dataset['dev_x'][n], 
                                'T', 'W', 
                                pos_outcome_spaces, 
                                pos_dataset['vocab_t.bos'], 
                                pos_dataset['vocab_w.eos'], 
                                pos_dataset['vocab_w.unk'], 
                                pos_template_factors)
    for inp, rv_e, rv_p, target in zip(pos_dataset['dev_x'][n], sorted(e.keys()), sorted(pred.keys()), pos_dataset['dev_y'][n]):
        table.append([inp, e[rv_e], pred[rv_p], target])
        
    print(tabulate(table, headers=['Input', 'Obs', 'Pred', 'Gold']))
    # print accuracy
    correct = sum(1 for i in range(len(pos_dataset['dev_y'][n])) if pos_dataset['dev_y'][n][i] == pred[f'T[{i+1:03d}]'])
    print(f"Accuracy: {correct}/{len(pos_dataset['dev_y'][n])} correct \n")
    print()


## Classification Report

We have already seen that the model does quite well in terms of classifying the POS-tags correctly.

Let's look at how well the POS-tagger performs for each tag individually.

We can use sklearn's classification report, which will compare each predicted tag to each gold tag and summarise these comparisons in a number of interesting ways. 

In [ ]:
def extract_pairs_for_classification_report(data_x, data_y, pred_assignments, hidden_rv, obs_rv):
    """
    For sklearn's classification report we need a long list of targets and predictions, 
    so this function collects those by iterating over the labelled data and the predicted assignemnts.

    data_x: obs seqs
    data_y: hidden seqs
    pred_assignments: predicted dicts for the observed sequences
    hidden_rv: name of hidden rv
    obs_rv: name of observed rv
    """
    h_true = []
    h_pred = []
    for x, y, pred_assignment in zip(data_x, data_y, pred_assignments):        
        x_ = [pred_assignment[rv] for rv in sorted(pred_assignment.keys()) if rv.startswith(obs_rv)]
        y_ = [pred_assignment[rv] for rv in sorted(pred_assignment.keys()) if rv.startswith(hidden_rv)]
        for inp, obs, pred, target in zip(x, x_, y_, y):            
            h_true.append(target)
            h_pred.append(pred)
    return h_true, h_pred

In [ ]:
# First we get the predicted assignments
dev_assignments = []
for obs_seq in pos_dataset['dev_x']:
    pred, e = tag_seq_from_text(obs_seq, 'T', 'W', pos_outcome_spaces, pos_dataset['vocab_t.bos'], pos_dataset['vocab_w.eos'], pos_dataset['vocab_w.unk'], pos_template_factors)
    dev_assignments.append({**e, **pred})


# Then we extract the pairs and report the results
h_true, h_pred = extract_pairs_for_classification_report(
    pos_dataset['dev_x'], 
    pos_dataset['dev_y'], 
    dev_assignments, 'T', 'W')
print(classification_report(h_true, h_pred, zero_division=0))

**EXERCISE - Analysis POS-tagging [1pt]**

Analyze the results you got in both the classification report, as well as the sentence level predictions. What do you observe? Where do we observe mistakes/low scores? Why? Feel free to consult https://universaldependencies.org/u/pos/ .

ANSWER: ...

**EXERCISE - Reconstruct Missing POS Tags [2pts]**

In this exercise, you'll take some dev instances and drop some POS tags in the middle, then use inference to reconstruct them. This tests whether the model can use context (both words and surrounding tags) to predict missing tags.

Complete the function `reconstruct_missing_tags()` below. The function should:

1. Take a sequence pair (words and tags) where some tags are missing (represented as `None`)
2. Build an HMM with evidence for the observed words and known tags
3. Use max-product inference to predict the missing tags
4. Return the reconstructed tag sequence

**Hint**: You'll need to condition on both the observed words AND the known tags, then infer the missing tags.


In [ ]:
def reconstruct_missing_tags(
    words: list,
    tags_with_gaps: list,  # List where some elements are None for missing tags
    hidden_rv: str,
    obs_rv: str,
    outcome_spaces: dict,
    h_bos: str,
    o_eos: str,
    o_unk: str,
    template_factors: dict
):
    """
    Reconstruct missing POS tags using MAP inference.
    
    Args:
        words: List of words (observed sequence)
        tags_with_gaps: List of tags where None indicates missing tags
        hidden_rv: name of hidden rv (e.g., 'T' for tags)
        obs_rv: name of observed rv (e.g., 'W' for words)
        outcome_spaces: dict mapping rv name to OutcomeSpace object
        h_bos: BOS for hidden sequence
        o_eos: EOS for observed sequence
        o_unk: UNK symbol for observed sequence
        template_factors: dict mapping rv name to table for the HMM's TabularCPDFactor objects
        
    Returns:
        List with missing tags filled in (None values replaced with predicted tags)
    """

    ### YOUR CODE HERE

    raise NotImplementedError("You need to implement this!")


Let's see how it performs:

In [ ]:
for i in range(10):
    test_words = pos_dataset['dev_x'][i]#[:15]  # First 15 words
    test_tags = pos_dataset['dev_y'][i]#[:15]   # First 15 tags

    # Skip if sentence is too short
    if len(test_words) < 3:
        continue

    # Mask every 2nd tag
    masked_tags = [tag if i % 2 != 0 else None 
                for i, tag in enumerate(test_tags)]

    reconstructed = reconstruct_missing_tags(
        test_words,
        masked_tags,
        'T', 'W',
        pos_outcome_spaces,
        pos_dataset['vocab_t.bos'],
        pos_dataset['vocab_w.eos'],
        pos_dataset['vocab_w.unk'],
        pos_template_factors
    )

    table = []
    for inp, mask, pred, gold in zip(test_words, masked_tags, reconstructed, test_tags):
        if mask is not None:
            table.append([inp, mask, pred, gold])
        else:
            table.append([inp, "___", pred, gold])
        
    print(tabulate(table, headers=['Input', 'Mask', 'Pred', 'Gold']))
    print()
    print("Accuracy on masked positions:")
    masked_positions = [i for i, tag in enumerate(masked_tags) if tag is None]
    correct = sum(1 for i in masked_positions if test_tags[i] == reconstructed[i])
    print(f"{correct}/{len(masked_positions)} correct \n")


## Exercise: Marginal Probability Queries and Fluency Ranking

**EXERCISE - Rank Fluent vs Disfluent Sentences - Code [1pt]**

In this exercise, you'll compute the marginal probability of text sequences (by marginalizing over tag sequences) and use this to rank fluent sentences over disfluent ones.

The marginal probability $P(X=x)$ of a word sequence is computed by summing over all possible tag sequences:
$$P(X=x) = \sum_{y} P(X=x, Y=y)$$

Complete the function `compute_text_probability()` and use it to rank sentence pairs.

__Hint__: You can use the `sum_product_variable_elimination()` function of week 5.


In [ ]:
def compute_text_probability(
    words: list,
    hidden_rv: str,
    obs_rv: str,
    outcome_spaces: dict,
    h_bos: str,
    o_eos: str,
    o_unk: str,
    template_factors: dict
):
    """
    Compute the marginal probability P(X=x) of a word sequence by marginalizing over all tag sequences.
    
    Args:
        words: List of words
        hidden_rv: name of hidden rv (e.g., 'T' for tags)
        obs_rv: name of observed rv (e.g., 'W' for words)
        outcome_spaces: dict mapping rv name to OutcomeSpace object
        h_bos: BOS for hidden sequence
        o_eos: EOS for observed sequence
        o_unk: UNK symbol for observed sequence
        template_factors: dict mapping rv name to table for the HMM's TabularCPDFactor objects
        
    Returns:
        Probability P(X=x)
    """
    ### YOUR CODE HERE

    raise NotImplementedError("You need to implement this!")



Now let's create some sentence pairs (fluent vs disfluent) and see if the model can rank them correctly:


In [ ]:
# Create sentence pairs: (fluent, disfluent)
# Disfluencies include: word order errors, missing words, extra words, wrong word choices
sentence_pairs = [
    ("Object-subject swap",
        ['the', 'president', 'drives', 'the', 'car'], 
        ['the', 'car', 'drives', 'the', 'president']),
    ("Displaced article",
        ['the', 'loan', 'may', 'be', 'extended'], 
        ['loan', 'may', 'be', 'extended', 'the']),
    ("Object-article swap",
        ['the', 'fees', 'would', 'apply', 'to', 'filings'], 
        ['fees', 'the', 'would', 'apply', 'to', 'filings']),
    ("Displaced verb",
        ['recent', 'industry', 'forecasts', 'for', '1990', 'indicate', 'a', 'slow', 'environment'], 
        ['recent', 'industry', 'forecasts', 'for', '1990', 'a', 'slow', 'environment', 'indicate']),
    ("Object-verb swap",
        ['this', 'does', 'not', 'sit', 'well', 'with', 'me'], 
        ['does','this', 'not', 'sit', 'well', 'with', 'me']),
    ("Displaced preposition",
        ['he', 'went', 'to', 'the', 'school'], 
        ['he', 'went', 'the', 'school', 'to']),
    ("Missing subject",
        ['talcott', 'led', 'a', 'team', 'of', 'researchers'], 
        ['led', 'a', 'team', 'of', 'researchers']),
    ("Verb-object swap",
        ['the', 'former', 'president', 'raised', 'controversial', 'issues'], 
        ['the', 'former', 'president', 'controversial', 'issues', 'raised']),
    ("Displaced subject",
        ['the', 'car', 'stopped'], 
        ['the', 'stopped', 'car']),
    ("Missing verb",
        ['i', 'kept', 'the', 'paper', 'alive'], 
        ['i', 'the', 'paper', 'alive']),
]

# Compute probabilities and compare
correct_rankings = 0
for i, (adjustment, fluent, disfluent) in enumerate(sentence_pairs, 1):
    # Compute probabilities
    prob_fluent = compute_text_probability(
        fluent, 'T', 'W', pos_outcome_spaces,
        pos_dataset['vocab_t.bos'], pos_dataset['vocab_w.eos'],
        pos_dataset['vocab_w.unk'], pos_template_factors
    )
    
    prob_disfluent = compute_text_probability(
        disfluent, 'T', 'W', pos_outcome_spaces,
        pos_dataset['vocab_t.bos'], pos_dataset['vocab_w.eos'],
        pos_dataset['vocab_w.unk'], pos_template_factors
    )
    
    is_correct = prob_fluent > prob_disfluent
    if is_correct:
        correct_rankings += 1
    
    print(f"\nPair {i}: {adjustment}")
    print(f"  Fluent:    {' '.join(fluent)}")
    print(f"  Disfluent: {' '.join(disfluent)}")
    print(f"  BF:  {prob_fluent/prob_disfluent:.5f}")
    print(f"  Ranking:   {'CORRECT' if is_correct else 'WRONG'}")

print("\n" + "=" * 80)
print(f"Correct rankings: {correct_rankings}/{len(sentence_pairs)}")
print(f"Accuracy: {correct_rankings/len(sentence_pairs)*100:.1f}%")


**EXERCISE - Rank Fluent vs Disfluent Sentences - Analysis [1pt]**

1. How well does the model distinguish fluent from disfluent sentences? What types of errors does it catch and which does it not catch?

ANSWER: ...

2. Why might some "disfluent" sentences actually have higher probability? 

ANSWER: ...



## Semi-Supervised Learning

Thus far we have seen that the model performs quite well on most tasks. Let's see what happens if the dataset is much smaller:

In [ ]:
# We take 1/10 of the training data
N = len(pos_dataset['training_x'])//10

pos_small_template_factors = estimate_parameters_hmm(
    zip(pos_dataset['training_x'][:N], pos_dataset['training_y'][:N]),    
    'T',
    'W',
    pos_outcome_spaces
)

# We use the same dev set as before
pos_small_bn = construct_hmm('T', 'W', pos_outcome_spaces, '-BOS-', pos_small_template_factors, 10)
pos_small_dev_assignments = []
for obs_seq in pos_dataset['dev_x'][:N]:
    pred, e = tag_seq_from_text(obs_seq, 'T', 'W', pos_outcome_spaces, pos_dataset['vocab_t.bos'], pos_dataset['vocab_w.eos'], pos_dataset['vocab_w.unk'], pos_small_template_factors)
    pos_small_dev_assignments.append({**e, **pred})

h_small_true, h_small_pred = extract_pairs_for_classification_report(
    pos_dataset['dev_x'], 
    pos_dataset['dev_y'], 
    pos_small_dev_assignments, 'T', 'W')
print(classification_report(h_small_true, h_small_pred, zero_division=0))

We see that the performance drops! We can improve our model using some combination of labeled and unlabeled data:

**EXERCISE - Semi-supervised Learning [1pts]**

Implement the semi-supervised learning method below. The method works as follows:

1. Estimate the parameters using the labeled data
2. Use the parameters to generate labels for the unlabeled data
3. Use the combined dataset to estimate the new parameters
4. Repeat steps 2 and 3 for a fixed number of times

Then use the resulting model to make predictions for the dev set and run the classification report so you can compare before and after. 

To sample new labels you will need to use Gibbs sampling:
- for the initial assignment you can use an arbitrary assignment (as `draw_initial_assignment` produces) or you can use a MAP assignment
- you can run the chain for a small number of iterations (e.g., 10)
- you can assume the Gibbs chain is mixed even after a small number of iterations (such as 10)
- take the last sample in the chain as the sequence that "completes" the unlabelled input sequence


In [ ]:
def semisupervised_estimation_hmm(
    labelled_data, unlabelled_data, dev_labelled, hidden_rv, obs_rv, outcome_spaces, 
    h_bos, h_eos, o_bos, o_eos, o_unk, 
    num_iterations, gibbs_num_iterations=10):
    """
    labelled_data: sequence pairs
    unlabelled_data: unlabelled sequences
    dev_labelled: heldout sequence pairs
    hidden_rv: name of hidden rv
    obs_rv: name of obs rv
    outcome_spaces: dict mapping rv name to OutcomeSpace object
    h_bos: BOS for hidden sequence
    h_eos: EOS for hidden sequence
    o_bos: BOS for observed sequence
    o_bos: EOS for observed sequence
    o_unk: UNK symbol for observed sequence
    num_iterations: for semi-supervised training
    gibbs_num_iterations: how many iterations of Gibbs sampling 
        (we only use the last sample, hence this is basically all burn-in, except for the last sample, which we keep)
    """
    # initialise tables with pseudo counts for smoothing
    val_h = outcome_spaces[hidden_rv]
    val_o = outcome_spaces[obs_rv]    

    labelled_data = list(labelled_data)
    dev_labelled = list(dev_labelled)
    
    parameters = estimate_parameters_hmm(
        labelled_data, 
        hidden_rv=hidden_rv, 
        obs_rv=obs_rv, 
        outcome_spaces=outcome_spaces
    )

    for k in range(num_iterations):  
        completed_data = []
        # complete unlabelled data
        print(f"{k+1}/{num_iterations} Labelling {len(unlabelled_data)} data points via Gibbs sampling")
        pbar = tqdm(unlabelled_data)
        for x in pbar:

            ### YOUR CODE HERE

            raise NotImplementedError("You need to implement this!")
        
        parameters = estimate_parameters_hmm(
            labelled_data + completed_data, 
            hidden_rv=hidden_rv, 
            obs_rv=obs_rv, 
            outcome_spaces=outcome_spaces
        )
        
        
    return parameters
    

In [ ]:
N = len(pos_dataset['training_x'])//10

new_parameters = semisupervised_estimation_hmm(
    zip(pos_dataset['training_x'][:N], pos_dataset['training_y'][:N]),    
    pos_dataset['training_x'][N:], 
    dev_labelled=zip(pos_dataset['dev_x'], pos_dataset['dev_y']),
    hidden_rv='T', 
    obs_rv='W', 
    outcome_spaces=pos_outcome_spaces, 
    h_bos=pos_dataset['vocab_t.bos'],
    h_eos=pos_dataset['vocab_t.eos'],
    o_bos=pos_dataset['vocab_w.bos'],
    o_eos=pos_dataset['vocab_w.eos'],
    o_unk=pos_dataset['vocab_w.unk'],
    num_iterations=1,
    gibbs_num_iterations=10
)

new_dev_assignments = []
for obs_seq in pos_dataset['dev_x'][:N]:
    pred, e = tag_seq_from_text(obs_seq, 'T', 'W', pos_outcome_spaces, pos_dataset['vocab_t.bos'], pos_dataset['vocab_w.eos'], pos_dataset['vocab_w.unk'], new_parameters)
    new_dev_assignments.append({**e, **pred})

h_true, h_pred = extract_pairs_for_classification_report(
    pos_dataset['dev_x'], 
    pos_dataset['dev_y'], 
    new_dev_assignments, 'T', 'W')
print(classification_report(h_true, h_pred, zero_division=0))

Great! We can see that semi-supervised learning greatly improves the performance accuracy even after one step!

# Summary

In this notebook, you learned about parameter learning in PGMs:

1. **HMMs**: A concrete example of a PGM that models sequences
2. **MLE Training**: Simple count-and-normalize procedure for learning parameters from fully observed data
3. **Testing**: Evaluating trained models using inference methods
4. **Semi-supervised**: Improve training using unlabeled samples and sampling


# Use of AI Tools

By submitting this notebook for grading you testify that:

* AI did not draft an earlier version of your work.
* You did not use AI-powered code completion.
* You did not implement algorithms suggested by an AI tool.
* AI did not revise a version of your work.
* You did not implement suggestions made by an AI tool.

_You_ in the sentences above refers to you and all your team members.
_AI_ refers to LM-based tools and assistants (e.g., ChatGPT, Gemini, UvA AI chat, etc.).

If you did make use of an AI tool, you should describe the uses you made of it below. Or indicate that no such tool was used.


**TYPE YOUR STATEMENT HERE**
